<h1> Electric Motor Dataset Analysis </h1>

<p>Introductory analysis for Kaggle <a href="https://www.kaggle.com/wkirgsn/electric-motor-temperature">Electric Motor Temperature</a> dataset. We'll see if we can parse out any useful information in a couple of notebooks and then hopefully create some sort of CLI program eventually.</p>